In [11]:
import os

In [12]:
os.chdir("../")
%pwd

'd:\\codes\\mlops\\Kidney-Disease-Classification'

In [13]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir : Path
    base_model_path : Path
    updated_base_model_path : Path
    
    params_image_size : list
    params_learning_rate : float
    params_include_top : bool
    params_weights : str
    params_classes : int
    
    

In [14]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.utils import read_yaml , create_direcories

In [15]:
class ConfigurationManager():
    def __init__(self, CONFIG_FILE_PATH = CONFIG_FILE_PATH , PARAMS_FILE_PATH = PARAMS_FILE_PATH):
           self.config = read_yaml(CONFIG_FILE_PATH)
           self.params = read_yaml(PARAMS_FILE_PATH)
           create_direcories([self.config.artifacts_root])
    def get_prepare_base_model_config(self)->PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        params = self.params
        create_direcories([config.root_dir])
        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir = config.root_dir,
            base_model_path = config.base_model_path,
            updated_base_model_path = config.updated_base_model_path,
            params_image_size = params.IMAGE_SIZE,
            params_learning_rate = params.LEARNING_RATE,
            params_include_top = params.INCLUDE_TOP,
            params_weights = params.WEIGHTS,
            params_classes = params.CLASSES,
        )
        return prepare_base_model_config


In [16]:
import os, sys
import tensorflow as tf
from src.cnnClassifier.exception.exception import customexception

In [17]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config
        
    def get_base_model(self):
        self.model = tf.keras.applications.ResNet50(
            input_shape= tuple(self.config.params_image_size),
            weights = self.config.params_weights,
            include_top= self.config.params_include_top
        )
        self.save_model(model = self.model , path = self.config.base_model_path)
        
    
    def prepare_full_model(self, model , classes , freeze_all , freeze_till , learning_rate):
        try:
            if freeze_all:
                for layer in model.layers:
                    layer.trainable = False
            else:
                for layer in model.layers[:freeze_till]:
                    layer.trainable = False
                for layer in model.layers[freeze_till:]:
                    layer.trainable = True
            flatten_in = tf.keras.layers.Flatten()(model.output)
            output = tf.keras.layers.Dense(units = classes, activation='softmax')(flatten_in)
            full_model = tf.keras.models.Model(
                inputs = model.input,
                outputs = output,
                )
            full_model.compile(
                optimizer=tf.keras.optimizers.Adam(lr=learning_rate),
                loss=tf.keras.losses.CategoricalCrossentropy(),
                metrics=['accuracy']
            )
            full_model.summary()
            return full_model
            
        except Exception as e:
            raise customexception(e, sys)
    
    def get_full_model(self):
        self.full_model = self.prepare_full_model(
            model = self.model,
            classes = self.config.params_classes,
            freeze_all = True,
            freeze_till = None,
            learning_rate = self.config.params_learning_rate
        )
        self.save_model(model = self.full_model, path = self.config.updated_base_model_path)
                        
    @staticmethod
    def save_model(model : tf.keras.Model, path : Path):
        model.save(path)

In [18]:
try:
    config_manager = ConfigurationManager()
    prepare_base_model_config = config_manager.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.get_full_model()
except Exception as e:
    raise customexception(e, sys)

[2024-12-31 23:33:11,032: INFO: utils: yaml file : config\config.yaml read successfully]


[2024-12-31 23:33:11,062: INFO: utils: yaml file : params.yaml read successfully]
[2024-12-31 23:33:11,066: INFO: utils: directory : artifacts created successfully]
[2024-12-31 23:33:11,068: INFO: utils: directory : artifacts\prepare_base_model created successfully]
[2024-12-31 23:33:14,396: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
[2024-12-31 23:33:15,210: WARNING: optimizer: `lr` is deprecated in Keras optimizer, please use `learning_rate` or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.Adam.]
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]      